In [7]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
import sklearn
import matplotlib.pyplot as plt

In [8]:
# linear regression methodology
class LR:
    
    def fit(self, X_train, y_train):
        # create vector of ones...
        ones = np.ones(shape=len(X_train))[..., None]
        #...and add to feature matrix
        X = np.concatenate((ones, X_train), 1)
        #calculate coefficients using closed-form solution
        self.coeffs = inv(X.transpose().dot(X)).dot(X.transpose()).dot(y_train)
        
    def predict(self, X_test):
        ones = np.ones(shape=len(X_test))[..., None]
        X_test = np.concatenate((ones, X_test), 1)
        y_hat = X_test.dot(self.coeffs)
        return y_hat


In [9]:
# cross validation methodology
def k_fold(k, df):
    n = len(df)
    cut = int(n/k)
    folds = []
    start = 0
    end = cut
    for i in range(0, k):
        fold = df[start: end]
        folds.append(fold)
        start += int(n/k)
        end += int(n/k)
    return folds

def mse(actual, predicted):
    return -(((actual - predicted)**2).mean())

def cv(folds, response):
    test_errors = []
    for i in range(0, len(folds)):
        X_train = pd.DataFrame()
        Y_train = pd.Series(dtype=float)
        for j in range(0, len(folds)):
            if i == j:
                X_test = folds[j].drop([response], axis=1)
                Y_test = folds[j][response]
            if i != j:
                X_train = X_train.append(folds[j].drop([response], axis=1))
                Y_train = Y_train.append(folds[j][response])
    
        model = LR()
        model.fit(X_train, Y_train.transpose())
        Y_pred = pd.DataFrame()
        Y_pred = model.predict(X_test)
        error = mse(Y_test, Y_pred) 
        test_errors.append(error)
        rmse = np.sqrt(-sum(test_errors)/len(folds))
        rss = (Y_test - Y_pred)**2
        rss = rss.sum()
        n = len(folds[j])
        p = len(X_train.columns)
        BIC = p * np.log(n) + n * np.log(rss/n)
    return rmse, BIC

In [10]:
def standardize(df): 
    #standardize only quantitative variables
    df_st = ((df.select_dtypes(float) - df.select_dtypes(float).mean()) / df.select_dtypes(float).std()) 

    #join the standardized quantites back with original df 
    df_st = df.select_dtypes(exclude=float).join(df_st)
    return df_st 

In [46]:
df = pd.read_csv("iowa_month_county.csv") 
df.set_index(["County", "Month-Year"], inplace=True)
df["Pack"] = df["Pack"].astype(float)
df["Population"] = df["Population"].astype(float)
df["Income Per Capita"] = df["Income Per Capita"].astype(float)
df = standardize(df)
df.head()

Pack  State Bottle Retail  Population  \
County Month-Year                                              
adair  01-2012    -0.371777            -0.385376   -0.425996   
       02-2012    -0.363708            -0.359456   -0.425996   
       03-2012    -0.378468            -0.399508   -0.425996   
       04-2012    -0.341077            -0.370412   -0.425996   
       05-2012    -0.356821            -0.348933   -0.425996   

                   Volume Sold (Gallons) Per Capita  Income Per Capita  \
County Month-Year                                                        
adair  01-2012                            -0.202512          -0.519043   
       02-2012                            -0.142834          -0.519043   
       03-2012                            -0.804345          -0.519043   
       04-2012                            -0.086595          -0.519043   
       05-2012                             0.075445          -0.519043   

                   Precincts     Votes  Republicans 2016  Democrats 2016  \
County Month-Year                                                          
adair  01-2012     -0.568508 -0.425356          0.443996       -0.403836   
       02-2012     -0.568508 -0.425356          0.443996       -0.403836   
       03-2012     -0.568508 -0.425356          0.443996       -0.403836   
       04-2012     -0.568508 -0.425356          0.443996       -0.403836   
       05-2012     -0.568508 -0.425356          0.443996       -0.403836   

                   Green 2016  ...     Black  Hispanic    Asian  Amerindian  \
County Month-Year              ...                                            
adair  01-2012      -0.839757  ... -0.626864 -0.630398 -0.49685   -0.254331   
       02-2012      -0.839757  ... -0.626864 -0.630398 -0.49685   -0.254331   
       03-2012      -0.839757  ... -0.626864 -0.630398 -0.49685   -0.254331   
       04-2012      -0.839757  ... -0.626864 -0.630398 -0.49685   -0.254331   
       05-2012      -0.839757  ... -0.626864 -0.630398 -0.49685   -0.254331   

                      Other  Median Age  Teen.births  \
County Month-Year                                      
adair  01-2012    -0.584086    0.908835    -0.343806   
       02-2012    -0.584086    0.908835    -0.343806   
       03-2012    -0.584086    0.908835    -0.343806   
       04-2012    -0.584086    0.908835    -0.343806   
       05-2012    -0.584086    0.908835    -0.343806   

                   Sexually.transmitted.infections  Unemployment  \
County Month-Year                                                  
adair  01-2012                           -0.682832     -0.995888   
       02-2012                           -0.682832     -0.995888   
       03-2012                           -0.682832     -0.995888   
       04-2012                           -0.682832     -0.995888   
       05-2012                           -0.682832     -0.995888   

                   Violent.crime  
County Month-Year                 
adair  01-2012          -0.91142  
       02-2012          -0.91142  
       03-2012          -0.91142  
       04-2012          -0.91142  
       05-2012          -0.91142  

[5 rows x 30 columns]

In [47]:
# MODEL with all p features
response = "Volume Sold (Gallons) Per Capita"
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

rmse: 1.0475749986037666


We're going to use the Backward Stepise approach to find our "most predictive model".  Because our model includes 29 features, the Best Subsets method is not feasible. 

We start by looking at the 5-fold cross validation estimated RMSE for each feature dropped.  Then, we take the model that (upon dropping a feature) has the lowest RMSE.  We continue until our approximated RMSE fails to improve.

In [48]:
# Best model with p - 1 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("State Bottle Retail", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                                1.046746
State Bottle Retail                                 1.007357
Population                                          1.096187
Income Per Capita                                   1.059220
Precincts                                           1.101184
Votes                                               1.022534
Republicans 2016                                    1.021002
Democrats 2016                                      1.020625
Green 2016                                          1.038474
Libertarians 2016                                   1.061670
At Least High School Diploma                        1.063870
At Least Bachelors's Degree                         1.049060
School Enrollment                                   1.059674
Median Earnings 2010                                1.047673
Children Under 6 Living in Poverty                  1.077488
Adults 65 and Older Living in Poverty               1.045852
Preschool.Enrollment.Rat

In [49]:
# Best model with p - 2 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("Votes", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                                1.085226
Population                                          1.036388
Income Per Capita                                   1.016598
Precincts                                           1.063717
Votes                                               0.987272
Republicans 2016                                    0.978120
Democrats 2016                                      0.977988
Green 2016                                          0.996705
Libertarians 2016                                   1.014530
At Least High School Diploma                        1.020446
At Least Bachelors's Degree                         0.988613
School Enrollment                                   1.013056
Median Earnings 2010                                1.011994
Children Under 6 Living in Poverty                  1.035132
Adults 65 and Older Living in Poverty               1.006012
Preschool.Enrollment.Ratio.enrolled.ages.3.and.4    1.002294
Poverty.Rate.below.feder

In [50]:
# Best model with p - 3 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("Republicans 2016", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                                1.078766
Population                                          1.169754
Income Per Capita                                   0.995209
Precincts                                           1.043434
Republicans 2016                                    0.960079
Democrats 2016                                      0.960154
Green 2016                                          0.972363
Libertarians 2016                                   0.993118
At Least High School Diploma                        0.999692
At Least Bachelors's Degree                         0.962252
School Enrollment                                   0.988054
Median Earnings 2010                                0.992410
Children Under 6 Living in Poverty                  1.009679
Adults 65 and Older Living in Poverty               0.987585
Preschool.Enrollment.Ratio.enrolled.ages.3.and.4    0.984768
Poverty.Rate.below.federal.poverty.threshold        1.011011
White                   

In [51]:
# Best model with p - 4 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("Green 2016", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                                1.042316
Population                                          1.124164
Income Per Capita                                   0.970012
Precincts                                           1.008944
Democrats 2016                                      0.977405
Green 2016                                          0.936807
Libertarians 2016                                   0.974388
At Least High School Diploma                        0.966913
At Least Bachelors's Degree                         0.948175
School Enrollment                                   0.959129
Median Earnings 2010                                0.955493
Children Under 6 Living in Poverty                  0.978246
Adults 65 and Older Living in Poverty               0.962155
Preschool.Enrollment.Ratio.enrolled.ages.3.and.4    0.955722
Poverty.Rate.below.federal.poverty.threshold        0.974159
White                                               0.950770
Black                   

In [52]:
# Best model with p - 5 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("Sexually.transmitted.infections", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                                1.007383
Population                                          1.090719
Income Per Capita                                   0.945545
Precincts                                           0.979769
Democrats 2016                                      0.942707
Libertarians 2016                                   0.940716
At Least High School Diploma                        0.945626
At Least Bachelors's Degree                         0.929579
School Enrollment                                   0.936564
Median Earnings 2010                                0.930337
Children Under 6 Living in Poverty                  0.946760
Adults 65 and Older Living in Poverty               0.936717
Preschool.Enrollment.Ratio.enrolled.ages.3.and.4    0.928887
Poverty.Rate.below.federal.poverty.threshold        0.948214
White                                               0.928581
Black                                               0.928960
Hispanic                

In [53]:
# Best model with p - 6 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("Preschool.Enrollment.Ratio.enrolled.ages.3.and.4", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                                0.990112
Population                                          1.070952
Income Per Capita                                   0.935624
Precincts                                           0.966639
Democrats 2016                                      0.931062
Libertarians 2016                                   0.925432
At Least High School Diploma                        0.932903
At Least Bachelors's Degree                         0.925535
School Enrollment                                   0.925315
Median Earnings 2010                                0.920698
Children Under 6 Living in Poverty                  0.936070
Adults 65 and Older Living in Poverty               0.925187
Preschool.Enrollment.Ratio.enrolled.ages.3.and.4    0.915083
Poverty.Rate.below.federal.poverty.threshold        0.940082
White                                               0.917351
Black                                               0.917785
Hispanic                

In [54]:
# Best model with p - 7 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("Other", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                            0.972982
Population                                      1.055586
Income Per Capita                               0.925459
Precincts                                       0.957879
Democrats 2016                                  0.917417
Libertarians 2016                               0.917397
At Least High School Diploma                    0.923639
At Least Bachelors's Degree                     0.916726
School Enrollment                               0.914533
Median Earnings 2010                            0.910854
Children Under 6 Living in Poverty              0.922055
Adults 65 and Older Living in Poverty           0.914378
Poverty.Rate.below.federal.poverty.threshold    0.926312
White                                           0.906652
Black                                           0.906934
Hispanic                                        0.906994
Asian                                           0.906679
Amerindian                     

In [55]:
# Best model with p - 8 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("Violent.crime", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                            0.968405
Population                                      1.065977
Income Per Capita                               0.917615
Precincts                                       0.954114
Democrats 2016                                  0.911857
Libertarians 2016                               0.911352
At Least High School Diploma                    0.912127
At Least Bachelors's Degree                     0.904125
School Enrollment                               0.904611
Median Earnings 2010                            0.905235
Children Under 6 Living in Poverty              0.912971
Adults 65 and Older Living in Poverty           0.906287
Poverty.Rate.below.federal.poverty.threshold    0.917765
White                                           0.898055
Black                                           0.899774
Hispanic                                        0.896456
Asian                                           0.904841
Amerindian                     

In [56]:
# Best model with p - 9 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("Amerindian", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                            0.954716
Population                                      1.034641
Income Per Capita                               0.908307
Precincts                                       0.939583
Democrats 2016                                  0.904156
Libertarians 2016                               0.913306
At Least High School Diploma                    0.913091
At Least Bachelors's Degree                     0.900074
School Enrollment                               0.899722
Median Earnings 2010                            0.903791
Children Under 6 Living in Poverty              0.906217
Adults 65 and Older Living in Poverty           0.900693
Poverty.Rate.below.federal.poverty.threshold    0.912076
White                                           0.893335
Black                                           0.895093
Hispanic                                        0.891644
Asian                                           0.900146
Amerindian                     

In [57]:
# Best model with p - 10 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("Asian", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                            0.936842
Population                                      0.962271
Income Per Capita                               0.892875
Precincts                                       0.898287
Democrats 2016                                  0.881136
Libertarians 2016                               0.896814
At Least High School Diploma                    0.889936
At Least Bachelors's Degree                     0.887260
School Enrollment                               0.888381
Median Earnings 2010                            0.886167
Children Under 6 Living in Poverty              0.878161
Adults 65 and Older Living in Poverty           0.884710
Poverty.Rate.below.federal.poverty.threshold    0.883623
White                                           0.882691
Black                                           0.882947
Hispanic                                        0.880723
Asian                                           0.875195
Median Age                     

In [58]:
# Best model with p - 11 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("Children Under 6 Living in Poverty", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                            0.927496
Population                                      0.958820
Income Per Capita                               0.883445
Precincts                                       0.890195
Democrats 2016                                  0.876174
Libertarians 2016                               0.891272
At Least High School Diploma                    0.878933
At Least Bachelors's Degree                     0.886454
School Enrollment                               0.874985
Median Earnings 2010                            0.876159
Children Under 6 Living in Poverty              0.869481
Adults 65 and Older Living in Poverty           0.879280
Poverty.Rate.below.federal.poverty.threshold    0.873160
White                                           0.880745
Black                                           0.880454
Hispanic                                        0.873925
Median Age                                      0.876972
Teen.births                    

In [59]:
# Best model with p - 12 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

df.drop("Unemployment", axis=1, inplace=True)
folds = k_fold(5,df)
out = cv(folds, response)
print("rmse:", out[0])

Pack                                            0.922290
Population                                      0.951639
Income Per Capita                               0.876367
Precincts                                       0.884756
Democrats 2016                                  0.872534
Libertarians 2016                               0.883134
At Least High School Diploma                    0.876429
At Least Bachelors's Degree                     0.878397
School Enrollment                               0.870568
Median Earnings 2010                            0.870170
Adults 65 and Older Living in Poverty           0.873668
Poverty.Rate.below.federal.poverty.threshold    0.870198
White                                           0.877258
Black                                           0.875199
Hispanic                                        0.872372
Median Age                                      0.872308
Teen.births                                     0.891525
Unemployment                   

In [60]:
# Best model with p - 13 features
errs = {}
for i in df:
    if (i != response):
        new_df = df.drop(i, axis=1)
        folds = k_fold(5, new_df)
        out = cv(folds, response)
        errs[i] = out[0]
        
errs = pd.Series(errs)
print(errs)

# df.drop("Unemployment", axis=1, inplace=True)
# folds = k_fold(5,df)
# out = cv(folds, response)
# print("rmse:", out[0])

Pack                                            0.918446
Population                                      0.945940
Income Per Capita                               0.876097
Precincts                                       0.880499
Democrats 2016                                  0.870929
Libertarians 2016                               0.884091
At Least High School Diploma                    0.873522
At Least Bachelors's Degree                     0.876420
School Enrollment                               0.866463
Median Earnings 2010                            0.867522
Adults 65 and Older Living in Poverty           0.870342
Poverty.Rate.below.federal.poverty.threshold    0.866650
White                                           0.876408
Black                                           0.872462
Hispanic                                        0.872674
Median Age                                      0.871351
Teen.births                                     0.887371
dtype: float64


None of the RMSE's in the output above are lower than the RMSE of .8663 in the previous step.  According to Backwards stepwise, this means we have arrived at our final model.  We stop dropping features and conclude that the model with p - 12 features is the best.

Most Predictive Model

- Number of features: 18
- Features: Pack, Population, Income Per Capita, Precints, Democrats 2016, Libertarians 2016, At Least High School Diploma, At Least Bachelor's Degree, School Enrollment, Median Earnings 2010, Adults 65 and Older Living in Poverty, Poverty.Rate.below.federal.poverty.threshold, White, Black, Hispanic, Median Age, Teen.births.
- Number of features dropped = 13
- Features dropped: State Bottle Retail, Votes, Republicans 2016, Green 2016, Sexually.transmitted.infections, Preschool.Enrollment.Ratio.enrolled.ages.3.and.4, Other, Violent.crime, Amerindian, Asian

- Estimated RMSE: .866

In [61]:
df.shape

(8294, 18)